### Preps & Introduction

So I've checked the public notebooks before I start and saw everbody is using **XGB**, **LGBM**, **CatBoost** and other shenanigans, decided to try CNN so here we go. Let's import the libraries.

In [ ]:

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

import tensorflow as tf
import os
import numpy as np
import pandas as pd
%matplotlib inline


In [ ]:
os.listdir('../input/tabular-playground-series-sep-2021/')


Loading the dataset

In [ ]:
train_data = pd.read_csv(
    '../input/tabular-playground-series-sep-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
submission_data = pd.read_csv(
    '../input/tabular-playground-series-sep-2021/sample_solution.csv')


### Pre-processing

Standard stuff.

In [ ]:
y = train_data['claim']
tmp = train_data.copy()
tmp['n_missing'] = train_data.isnull().sum(axis=1)
X = tmp.drop(['claim', 'id'], axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=0)

In [ ]:
imputer = SimpleImputer(strategy='mean')
imp_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imp_X_val = pd.DataFrame(imputer.transform(X_val))
imp_X_train.columns = X_train.columns
imp_X_val.columns = X_val.columns

In [ ]:
test_tmp = test_data.copy().drop('id', axis=1)
test_tmp['n_missing'] = test_data.isnull().sum(axis=1)

imp_X_test = pd.DataFrame(imputer.fit_transform(test_tmp))
imp_X_test.columns = test_tmp.columns

In [ ]:
imp_X_train

In [ ]:
y_train

In [ ]:
imp_X_test

In [ ]:
print(imp_X_train.shape)
print(y_train.shape)
print(imp_X_test.shape)


In [ ]:
X_train_expanded = tf.expand_dims(imp_X_train, axis=-1)
X_val_expanded = tf.expand_dims(X_val, axis=-1)

### Creating CNN model.

There's likely a better approach. 🤷‍♂️

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Input(shape=(imp_X_train.shape[1], 1,)))

model.add(tf.keras.layers.Conv1D(128, kernel_size=1,
          activation='relu', padding='same'))
model.add(tf.keras.layers.Conv1D(
    128, kernel_size=1, activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=1, strides=1))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv1D(256, kernel_size=1,
          activation='relu', padding='same'))
model.add(tf.keras.layers.Conv1D(
    128, kernel_size=1, activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=1, strides=1))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv1D(512, kernel_size=1,
          activation='relu', padding='same'))
model.add(tf.keras.layers.Conv1D(
    128, kernel_size=1, activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=1, strides=1))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(512, activation='relu', activity_regularizer=tf.keras.regularizers.l2(
    0.00001), bias_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(512, activation='relu', activity_regularizer=tf.keras.regularizers.l2(
    0.00001), bias_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(512, activation='relu', activity_regularizer=tf.keras.regularizers.l2(
    0.00001), bias_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Examine our model.

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)


Let's build our model and train it! I am using a high epoch but should stop if things go down.

In [ ]:
auc = tf.keras.metrics.AUC(name='aucroc')
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', auc])


In [ ]:
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=5, verbose=1, restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.3, patience=3, verbose=1, min_delta=1e-4
)

callbacks = [earlystopping, reduce_lr]

history = model.fit(x=X_train_expanded, y=y_train, batch_size=512, shuffle=True,
                    epochs=30, validation_data=(X_val_expanded, y_val), callbacks=callbacks)


In [ ]:
def vis_training(history_list, start=1):
    loss = np.concatenate([h.history["loss"] for h in history_list])
    val_loss = np.concatenate([h.history["val_loss"] for h in history_list])
    acc = np.concatenate([h.history["accuracy"] for h in history_list])
    val_acc = np.concatenate([h.history["val_accuracy"] for h in history_list])

    epoch_range = range(1, len(loss) + 1)

    plt.figure(figsize=[12, 6])
    plt.subplot(1, 2, 1)
    plt.plot(epoch_range[start - 1:], loss[start - 1:], label="Training Loss")
    plt.plot(epoch_range[start - 1:],
             val_loss[start - 1:], label="Validation Loss")
    plt.xlabel("Epoch")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epoch_range[start - 1:], acc[start - 1:],
             label="Training Accuracy")
    plt.plot(
        epoch_range[start - 1:], val_acc[start -
                                         1:], label="Validation Accuracy"
    )
    plt.xlabel("Epoch")
    plt.legend()

    plt.show()


I like plots. 🧑‍💻

In [ ]:
vis_training([history])


Creating the submission thingy.

In [ ]:
X_test_extended = tf.expand_dims(imp_X_test, axis=-1);

In [ ]:
sub = pd.DataFrame()
sub['id'] = test_data['id']
sub['claim'] = model.predict(X_test_extended)
sub = sub.set_index('id')

In [ ]:
sub.head()

In [ ]:
sub.shape


In [ ]:
sub.to_csv('submission.csv')
